In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np

In [8]:
stats = pd.read_csv(os.path.join("csv/MLB DB V1 CSV.csv"))
stats.rename(columns={"O/U Odds": "O/U_Odds"}, inplace= True)

stats.FIELD[stats.FIELD == 'HOME'] = 1
stats.FIELD[stats.FIELD == 'AWAY'] = 0

stats = stats.drop(columns = ["Team", "Season", "Date", "Vs", "Score", "Away Starter", "Home Starter", "BAL Line", "O/U", "O/U Result", "Unnamed: 9", "Margin", "Runs Scr", "Runs Alw"])

stats= stats.dropna()

stats.head()

,FIELD,Game Result,Side Line,O/U Line,O/U_Odds
0,1,L,-155,8.5,-106
1,1,W,-155,8.5,-116
2,1,W,-145,8.0,-106
3,1,W,-120,9.0,107
4,0,W,176,9.0,-107


In [9]:
stats.drop(stats[stats["O/U_Odds"] == "9 -"].index, inplace=True)
stats.drop(stats[stats["O/U_Odds"] == "7 -"].index, inplace=True)
stats.drop(stats[stats["O/U_Odds"] == "8 -"].index, inplace=True)
stats.drop(stats[stats["O/U_Odds"] == "10 -"].index, inplace=True)
stats.drop(stats[stats["O/U_Odds"] == "11 -"].index, inplace=True)
stats.drop(stats[stats["O/U_Odds"] == "6 -"].index, inplace=True)

In [10]:
stats.FIELD[stats.FIELD == 'HOME'] = 1
stats.FIELD[stats.FIELD == 'AWAY'] = 0

In [11]:
# Assign X (data) and y (target)
X = stats.drop("Game Result", axis=1)
y = stats["Game Result"]
print(X.shape, y.shape)

(4690, 4) (4690,)


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [13]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression()

In [14]:
classifier.fit(X_train, y_train)

LogisticRegression()

In [15]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.5769121410292863
Testing Data Score: 0.5797101449275363


In [16]:
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   ['L' 'L' 'W' 'W' 'L' 'W' 'W' 'W' 'W' 'L']
First 10 Actual labels: ['L', 'W', 'W', 'W', 'W', 'L', 'W', 'W', 'W', 'W']


In [17]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

,Prediction,Actual
0,L,L
1,L,W
2,W,W
3,W,W
4,L,W
...,...,...
1168,W,W
1169,L,L
1170,W,W
1171,L,L


In [18]:
stats.to_csv(r'C:\Users\Jackson\Documents\Bootcamp\Final Project\indiana_data_farmers\Stats.csv', index = False) 

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=4))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [20]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=0
)

In [21]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

1173/1173 - 0s - loss: 0.6806 - accuracy: 0.5780
Normal Neural Network - Loss: 0.6805681623359674, Accuracy: 0.5780051350593567


In [23]:
import pickle

with open('mlb_betting.pkl', 'wb') as file:
    pickle.dump(classifier, file)

In [24]:
def predict_mpg(config, model):
    
    if type(config) == dict:
        df = pd.DataFrame(config)
    else:
        df = config
    
    y_pred = classifier.predict(X_test)
    return y_pred

In [39]:
odds = {
    'FIELD': [1],
    'Side Line': [-500],
    'O/U Line': [7.5],
    'O/U_Odds': [-300]}

predict_mpg(odds, classifier)

In [57]:
new_data = np.array([[1, 1, 6,-150]])
print(f"Predicted class: {model.predict_classes(new_data)}")


Predicted class: [1]
